# Lab 3 - The Matrix Form and The Duality Theory

<b>Information on group members:</b><br>
1) 156071, Martyna Stasiak <br>
2) 156062, Maria Musiał

In [164]:
from pulp import *  
import numpy as np
import pandas as pd
import itertools
from itertools import combinations
from IPython.display import display

# 1) The Matrix Form - Fundamental Insight (finish this part to get 3.0)

1.1) Given is the below (primal) linear problem:

primal problem:<br>
MAXIMIZE<br>
4*x1 + 2*x2 + 3*x3 <br>

SUBJECT TO<br>
1_constraint: x1 + x2 <= 10<br>
2_constraint: 2*x2 + x3 <= 12<br>
3_constraint: 3*x1 + 2*x3 <= 15<br>
4_constraint: x1 + x2 + x3 <= 20<br>

VARIABLES<br>
x1 Continuous<br>
x2 Continuous<br>
x3 Continuous<br>

1.2) Use the PuLP library to solve the above problem. Identify the optimal solution: the values for basic variables and the corresponding value for the objective function.

In [165]:
### Model here:
model = LpProblem("Lab3TaskA", LpMaximize)
# Variables
x1 = LpVariable(name="x1", lowBound=0) #do they have to be nonnegative??????????
x2 = LpVariable(name="x2", lowBound=0)
x3 = LpVariable(name="x3", lowBound=0)

# Constraints
# x1 + x2 <= 10
model += (x1 + x2 <= 10, "#1 constraint")
# 2*x2 + x3 <= 12
model += (2*x2 + x3 <= 12, "#2 constraint")
# 3*x1 + 2*x3 <= 15
model += (3*x1 + 2*x3 <= 15, "#3 constraint")
# x1 + x2 + x3 <= 20
model += (x1 + x2 + x3 <= 20, "#4 constraint")

# Objective
obj_func = 4*x1 + 2*x2 + 3*x3
model += obj_func

model


Lab3TaskA:
MAXIMIZE
4*x1 + 2*x2 + 3*x3 + 0
SUBJECT TO
#1_constraint: x1 + x2 <= 10

#2_constraint: 2 x2 + x3 <= 12

#3_constraint: 3 x1 + 2 x3 <= 15

#4_constraint: x1 + x2 + x3 <= 20

VARIABLES
x1 Continuous
x2 Continuous
x3 Continuous

In [166]:
### Solve here
status = model.solve()
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}\n")
print('Constraints:')
for name, constraint in model.constraints.items():  print(f"{name}: {constraint.value()}")
print('\nVariables & their values:')
for var in model.variables():  print(f"{var.name}: {var.value()}")


status: 1, Optimal
objective: 31.428571379999998

Constraints:
#1_constraint: 0.0
#2_constraint: 5.999999985739635e-08
#3_constraint: -7.999999906971311e-08
#4_constraint: -9.14285714

Variables & their values:
x1: 4.4285714
x2: 5.5714286
x3: 0.85714286


1.3) In this exercise, you are asked to identify ALL basic (feasible and not) solutions to the above problem. We will do this naively. Specifically, you are asked to use the fundamental insight to build a final simplex tableau for each possible base. Therefore, you need first to initialize the data: c, b, A matrixes, and it is suggested to initialize the auxiliary matrix M defined as M = A + (concatenate) I (identity matrix). Note that the problem should be formulated in the augmented form. Then, you have to iterate over each possible base B, compute B-1, and other relevant parts for the simplex tableau.<br><br>
a) Identify the optimal solution using the optimality condition; print it (Z value and values for basic variables); compare thus derived solution with the optimum found using the PuLP library (obviously, both solutions should be the same). <br>
b) Count the number of feasible and infeasible solutions. How many (all) basic solutions to the problem can be identified? <br><br>
It is suggested to use the NumPy library for performing matrix operations. 

### Augmented form:
MAXIMIZE<br>
Z -4*x1 - 2*x2 - 3*x3 <br>

SUBJECT TO<br>
1) x1 + x2 + x4 = 10<br>
2) 2*x2 + x3 + x5 = 12<br>
3) 3*x1 + 2*x3 + x6 = 15<br>
4) x1 + x2 + x3 + x7 = 20<br>

In [167]:
### Initialize the data here:
### TODO
c = np.array([4, 2, 3]) #coefficients in the objective function
b = np.array([10, 12, 15, 20]).reshape(-1,1) #right-hand side of the constraints
A = np.array([[1, 1, 0], [0, 2, 1], [3, 0, 2], [1, 1, 1]]) #coefficients in the constraints
I = np.eye(4) # identity matrix
M = np.concatenate((A, I), axis=1) # The matrix M
base = np.array([3, 4, 5, 6]) # The base corresponds to the indices of the identity matrix columns in M


<b> Important note: the below is just a proposition. You can solve the problem in your own way. </b>

You can define an auxiliary method constructing a final simplex tableau for a given base.  Here, "base" is a list of columns (integers) for the base. Note that the functions in python can return multiple objects and you can use this functionality to return<br>
- the inversed base<br>
- coefficients in the row 0 for slack variables<br>
- right side values (except the objective function value)<br>
- the objective function value<br>
- the coefficients for decision variables in row 0 <br>
- the coefficients for decision variables in rows 1+<br>

Note that if BI cannot be built (it is possible), the method may return None in order to notify the executive method about this exception. 



In [168]:
#more like get a table :D
def getFinalTableau(base, c, b, A, M): 
    c_extended = np.concatenate((c, np.zeros(4)))
    c_b = c_extended[base]
    B = M[:, base]
    if B.shape[0] != B.shape[1] or np.linalg.matrix_rank(B) != B.shape[0]:
        return None
    else:
        B_inv = np.linalg.inv(B)
    x_b = B_inv @ b
    
    row0 = np.concatenate(([1], (c_b @ B_inv @ A - c), (c_b @ B_inv), (c_b @ B_inv @ b)), axis = 0)
    row1 = np.hstack((np.zeros((B_inv.shape[0], 1)), B_inv @ A, B_inv, B_inv @ b))
    tableau = np.vstack((row0, row1))
    
    #variables from row 1+ from the tableau
    Z_DecVar = c_b @ B_inv @ A - c
    Z_SlackVar = c_b @ B_inv
    Z_RS = c_b @ B_inv @ b
    x_DecVar = B_inv @ A
    x_SlackVar = B_inv
    x_RS = B_inv @ b
    
    return tableau, Z_DecVar, Z_SlackVar, Z_RS, x_DecVar, x_SlackVar, x_RS


In [186]:
#
def simplex_solution(c, b, A, M):
    all_bases = list(combinations(range(M.shape[1]), b.shape[0]))  # All combinations of columns for possible bases
    feasible_solutions = 0
    infeasible_solutions = 0
    optimal_solution_found = False
    treshold = -10**(-15) #due to the computation i set a small treshold equal to decimal num in model
    optimal_solution = {}

    for base in all_bases:
        #calculating a tableau for the curr base
        tableau_result = getFinalTableau(np.array(base), c, b, A, M)
        
        #if b is not invertible we add infisible solution ?dowe
        if tableau_result is None:
            #infeasible_solutions += 1
            continue
        
        #when B is invertible we create a tableau
        tableau, Z_DecVar, Z_SlackVar, Z_RS, x_DecVar, x_SlackVar, x_RS = tableau_result
        
        #feasibility check -> right sides of the basic variables hav to be nonnegative
        if min(x_RS) >= treshold:
            feasible_solutions += 1 #we increment num of feasible solutions
            print("\nFeasible solution found:")
            print("Objective value (Z) =", Z_RS[0]) #printing value of feasible solution

            #displaying values of dec var for feasible solution
            for i, var_index in enumerate(base):
                print(f"x{var_index} =", x_RS[i][0])

            #optimality Check -> all variables in row0 have to be nonnegative
            if min(Z_DecVar.min(), Z_SlackVar.min()) >= treshold:
                print(f"This optimal solution was found with objective value of {Z_RS[0]}.")
                optimal_solution_found = True
                #break  #we finish iterating after finding optimal solution
                optimal_solution["Objective Value"] = Z_RS[0]
                for i, var_index in enumerate(base):
                    optimal_solution[f"x{var_index}"] = x_RS[i][0]

        else:
            infeasible_solutions += 1

    #Sumarry of the results
    print("\nSummary:")
    print("Number of feasible solutions:", feasible_solutions)
    print("Number of infeasible solutions:", infeasible_solutions)
    print("Total number of base solutions checked:", feasible_solutions + infeasible_solutions)
    if optimal_solution_found:
        print("An optimal solution was found.")
        solution_df = pd.DataFrame([optimal_solution])
        return solution_df
    else:
        print("No optimal solution was found.")


In [188]:
simplex_solution(c, b, A, M)


Feasible solution found:
Objective value (Z) = 31.428571428571427
x0 = 4.428571428571427
x1 = 5.571428571428572
x2 = 0.8571428571428559
x6 = 9.142857142857144
This optimal solution was found with objective value of 31.428571428571427.

Feasible solution found:
Objective value (Z) = 30.0
x0 = 5.0
x1 = 5.0
x4 = 2.0
x6 = 10.0

Feasible solution found:
Objective value (Z) = 28.0
x0 = 4.0
x1 = 6.0
x5 = 3.0
x6 = 10.0

Feasible solution found:
Objective value (Z) = 20.0
x0 = 5.0
x3 = 5.0
x4 = 12.0
x6 = 15.0

Feasible solution found:
Objective value (Z) = 27.0
x1 = 2.25
x2 = 7.5
x3 = 7.75
x6 = 10.25

Feasible solution found:
Objective value (Z) = 12.0
x1 = 6.0
x3 = 4.0
x5 = 15.0
x6 = 14.0

Feasible solution found:
Objective value (Z) = 22.5
x2 = 7.5
x3 = 10.0
x4 = 4.5
x6 = 12.5

Feasible solution found:
Objective value (Z) = 0.0
x3 = 10.0
x4 = 12.0
x5 = 15.0
x6 = 20.0

Summary:
Number of feasible solutions: 8
Number of infeasible solutions: 23
Total number of base solutions checked: 31
An opt

,Objective Value,x0,x1,x2,x6
0,31.428571,4.428571,5.571429,0.857143,9.142857


## 2) The Duality Theory (finish this part + part 1 + to get 5.0)

2.1) Model the dual problem to the above solved primal one, using the PuLP library. Then, solve it and compare the derived optimum with the optimum for the primal problem. Are they equal?

### Primal problem:
MAXIMIZE Z<br>
4*x1 + 2*x2 + 3*x3 <br>

SUBJECT TO<br>
1_constraint: x1 + x2 <= 10<br>
2_constraint: 2*x2 + x3 <= 12<br>
3_constraint: 3*x1 + 2*x3 <= 15<br>
4_constraint: x1 + x2 + x3 <= 20<br>

VARIABLES<br>
x1 Continuous<br>
x2 Continuous<br>
x3 Continuous<br>

### Dual problem:
MINIMIZE W<br>
10*y1 + 12*y2 + 15*y3 + 20*y4 <br>

SUBJECT TO<br>
1_constraint: y1 + 3*y3 + y4 >= 4<br>
2_constraint: y1 + 2*y2 + y4 >= 2<br>
3_constraint: y2 + 2*y3 + y4 >= 3<br>

VARIABLES<br>
y1 Continuous<br>
y2 Continuous<br>
y3 Continuous<br>
y4 Continuous<br>


In [ ]:
### Model here:
DualModel = LpProblem("DualProblem", LpMinimize)
# Variables
y1 = LpVariable(name="y1", lowBound=0) 
y2 = LpVariable(name="y2", lowBound=0)
y3 = LpVariable(name="y3", lowBound=0)
y4 = LpVariable(name="y4", lowBound=0)

# Constraints
# y1 + 3*y3 + y4 >= 4
DualModel += (y1 + 3*y3 + y4 >= 4, "#1 constraint")
# y1 + 2*y2 + y4 >= 2
DualModel += (y1 + 2*y2 + y4 >= 2, "#2 constraint")
# 3*x1 + 2*x3 <= 15
DualModel += (y2 + 2*y3 + y4 >= 3, "#3 constraint")

# Objective function
obj_func = 10*y1 + 12*y2 + 15*y3 + 20*y4
DualModel += obj_func

DualModel


DualProblem:
MINIMIZE
10*y1 + 12*y2 + 15*y3 + 20*y4 + 0
SUBJECT TO
#1_constraint: y1 + 3 y3 + y4 >= 4

#2_constraint: y1 + 2 y2 + y4 >= 2

#3_constraint: y2 + 2 y3 + y4 >= 3

VARIABLES
y1 Continuous
y2 Continuous
y3 Continuous
y4 Continuous

In [172]:
### Solve here:
status = DualModel.solve()
print(f"status: {DualModel.status}, {LpStatus[DualModel.status]}")
print(f"objective: {DualModel.objective.value()}\n")
print('Constraints:')
for name, constraint in DualModel.constraints.items():  print(f"{name}: {constraint.value()}")
print('\nVariables & their values:')
for var in DualModel.variables():  print(f"{var.name}: {var.value()}")


status: 1, Optimal
objective: 31.42857072

Constraints:
#1_constraint: -1.2999999965401798e-07
#2_constraint: -9.99999993922529e-09
#3_constraint: -8.999999989711682e-08

Variables & their values:
y1: 0.57142857
y2: 0.71428571
y3: 1.1428571
y4: 0.0


2.2) This exercise is based on the exercise 1.3 (copy & paste solution). Here, you are asked to iterate over all basic solutions (as in 1.3) and store them along with their complementary dual solutions. Solutions should be stored in the PRIMAL_DUAL_SOLUTIONS list and sorted according to the objective value Z = W. Analyze their optimality and feasibility. Finally, you have to display all basic solutions wlong with their complementary solutions (you can use the provided piece of code written using the pandas library). <br><br>

PRIMAL_DUAL_SOLUTIONS is defined as a table consisting of n rows, where n is the number of basic solutions to the problem, and 21 columns. The columns are defined as follows:<br>
Col. 1: The objective value Z<br>
Col. 2-4: The values for decision variables (primal solution)<br>
Col. 5-8: The values for slack variables (primal solution)<br>
Col. 9: P_F = Y or N, Y/N = primal solution is feasible/infeasible<br>
Col. 10: P_O = Y or N, Y/N = primal solution is optimal/is not optimal<br>
Col. 11: P_STATE = -/suboptimal/superoptimal/optimal; depends on P_F and P_O (primal)<br>
Col. 12: D_STATE = -/suboptimal/superoptimal/optimal; depends on D_F and D_O (dual)<br>
Col. 13: D_F = Y or N, Y/N = dual solution is feasible/infeasible<br>
Col. 14: D_O = Y or N, Y/N = dual solution is optimal/is not optimal<br>
Col. 15-18: The values for decision variables (dual solution)<br>
Col. 19-21: The values for surplus variables (dual solution)<br><br>

Reminder: sort solutions according to Z; analyze how their states change with the increase of Z.

In [ ]:
def dual_simplex_solution(c, b, A, M):
    PrimalDualSolutions = []  #list to store all the solutions
    all_bases = list(combinations(range(M.shape[1]), b.shape[0]))  # All combinations of columns for possible bases
    treshold = -10**(-15) #due to the computation i set a small treshold equal to decimal num in model
    

    for base in all_bases:
        #calculating a tableau for the curr base
        tableau_result = getFinalTableau(np.array(base), c, b, A, M)
        
        #if b is not invertible we add infisible solution ?dowe
        if tableau_result is None:
            #infeasible_solutions += 1
            continue
        
        #when B is invertible we create a tableau
        tableau, Z_DecVar, Z_SlackVar, Z_RS, x_DecVar, x_SlackVar, x_RS = tableau_result
        
        row = [0] * 21 #creating a row with 21 fields to be filled with actual values
        
        feasible_solution = min(x_RS) >= treshold
        optimal_solution = min(Z_DecVar.min(), Z_SlackVar.min()) >= treshold
        
        #determining the state of the primal and dual solutions
        if feasible_solution and optimal_solution:
            primal_state = "optimal"
            dual_state = "optimal"
        elif feasible_solution and not optimal_solution:
            primal_state = "suboptimal"
            dual_state = "superoptimal"
        elif not feasible_solution and optimal_solution:
            primal_state = "superoptimal"
            dual_state = "suboptimal"
        else:
            primal_state = "-"
            dual_state = "-"
        
        #column 0 filled with the objective value
        row[0] = Z_RS[0]
        
        #columns for the primal variables
        for id, value in zip(base, x_RS):
            row[id + 1] = value[0]
        
        #primal feasibility and optimality
        row[8] = 'Y' if feasible_solution else 'N'
        row[9] = 'Y' if optimal_solution else 'N'
        
        #primal and dual states
        row[10] = primal_state
        row[11] = dual_state
        
        #dual feasibility and optimality
        row[12] = row[9]
        row[13] = row[8]
        
        #dual decision vars
        for i in range(len(Z_SlackVar)):
            row[14 + i] = Z_SlackVar[i]
            
        #dual surplus vars
        for i in range(len(Z_DecVar)):
            row[18 + i] = Z_DecVar[i]    
        
        PrimalDualSolutions.append(row)
        
    PrimalDualSolutions.sort(key=lambda x: x[0])
        
        
### DISPLAY STORED PAIRS OF SOLUTIONS
    df = pd.DataFrame(PrimalDualSolutions, columns = ["Z", "x1", "x2", "x3", "slack1", "slack2", "slack3", "slack4", "P_F", "P_O", "P_STATE", "D_STATE", "D_F", "F_O",
                                                    "y1", "y2", "y3", "y4", "sur1", "sur2", "sur3"])
    
    return df



X = dual_simplex_solution(c, b, A, M)
display(X.style.set_properties(**{
            'width': '230px',
            'max-width': '230px',
        }))



,Z,x1,x2,x3,slack1,slack2,slack3,slack4,P_F,P_O,P_STATE,D_STATE,D_F,F_O,y1,y2,y3,y4,sur1,sur2,sur3
0,-4.000000,0.000000,10.000000,-8.000000,0.000000,0.000000,31.000000,18.000000,N,N,-,-,N,N,-4.000000,3.000000,0.000000,0.000000,-8.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,10.000000,12.000000,15.000000,20.000000,Y,N,suboptimal,superoptimal,N,Y,0.000000,0.000000,0.000000,0.000000,-4.000000,-2.000000,-3.000000
2,12.000000,0.000000,6.000000,0.000000,4.000000,0.000000,15.000000,14.000000,Y,N,suboptimal,superoptimal,N,Y,0.000000,1.000000,0.000000,0.000000,-4.000000,0.000000,-2.000000
3,17.500000,10.000000,0.000000,-7.500000,0.000000,19.500000,0.000000,17.500000,N,N,-,-,N,N,-0.500000,0.000000,1.500000,0.000000,0.000000,-2.500000,0.000000
4,20.000000,5.000000,0.000000,0.000000,5.000000,12.000000,0.000000,15.000000,Y,N,suboptimal,superoptimal,N,Y,0.000000,0.000000,1.333333,0.000000,0.000000,-2.000000,-0.333333
5,20.000000,0.000000,10.000000,0.000000,0.000000,-8.000000,15.000000,10.000000,N,N,-,-,N,N,2.000000,0.000000,0.000000,0.000000,-2.000000,0.000000,-3.000000
6,22.500000,0.000000,0.000000,7.500000,10.000000,4.500000,0.000000,12.500000,Y,N,suboptimal,superoptimal,N,Y,0.000000,0.000000,1.500000,0.000000,0.500000,-2.000000,0.000000
7,24.000000,-3.000000,0.000000,12.000000,13.000000,0.000000,0.000000,11.000000,N,N,-,-,N,N,0.000000,0.333333,1.333333,0.000000,0.000000,-1.333333,0.000000
8,27.000000,0.000000,2.250000,7.500000,7.750000,0.000000,0.000000,10.250000,Y,N,suboptimal,superoptimal,N,Y,0.000000,1.000000,1.000000,0.000000,-1.000000,0.000000,0.000000
9,28.000000,4.000000,6.000000,0.000000,0.000000,0.000000,3.000000,10.000000,Y,N,suboptimal,superoptimal,N,Y,4.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,-4.000000
